<a href="https://colab.research.google.com/github/Sarvesh1814/US-Book-Recommendation-System/blob/main/Content_based_RS_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

### Mount drive and load datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Base_path ='/content/drive/MyDrive/Data/RS-3'

#### Books with ISBN

In [ ]:
Book_ISBN = pd.read_csv(Base_path+"/Books.csv")

In [ ]:
Book_ISBN.rename(columns = {'Book-Title':'Book_Title'}, inplace = True)

#### Books with Description

In [ ]:
Books = pd.read_csv(Base_path+"/Books_Final.csv")

In [ ]:
Books.head()

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description
0,0,0,Footfall,en,706,['Fiction'],"Hostile extraterrestrials, known as the Fithp,..."
1,1,1,The Reader,en,224,['Fiction'],Hailed for its coiled eroticism and the moral ...
2,2,2,Hidden Star (The Star Series),en,234,['Fiction'],Eleven-year-old Nolitye's granny used to say: ...
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,['Fiction'],This New York Times bestseller of a troubled f...
4,4,4,The Hours: A Novel,en,295,['Fiction'],Michael Cunningham brings together his Pulitze...


In [ ]:
df=Books

### Rating Data

In [ ]:
Ratings = pd.read_csv(Base_path+"/Ratings.csv")

In [ ]:
Ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### NLP preprocessing on Description Text

In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
def clean_text(text):
    """
    Clean the text by removing punctuation, stopwords, and converting to lowercase.
    """
    # Remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    
    # Convert to lowercase
    text = text.lower()

    # Remove non-English characters
    text = re.sub(r"[^\x00-\x7F]+", "", text)
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    
    # Perform lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join the words back into a single string
    cleaned_text = " ".join(words)
    
    return cleaned_text

In [ ]:
df["category"] = df["categories"].apply(clean_text)

In [ ]:
df["category"].head()

0    fiction
1    fiction
2    fiction
3    fiction
4    fiction
Name: category, dtype: object

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description,category
0,0,0,Footfall,en,706,['Fiction'],"Hostile extraterrestrials, known as the Fithp,...",fiction
1,1,1,The Reader,en,224,['Fiction'],Hailed for its coiled eroticism and the moral ...,fiction
2,2,2,Hidden Star (The Star Series),en,234,['Fiction'],Eleven-year-old Nolitye's granny used to say: ...,fiction
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,['Fiction'],This New York Times bestseller of a troubled f...,fiction
4,4,4,The Hours: A Novel,en,295,['Fiction'],Michael Cunningham brings together his Pulitze...,fiction


In [ ]:
# Replace all non-letters with spaces # Search for all non-letters
# define the function to remove non-alphabetic characters
def remove_non_alphabetic(text):
    return re.sub("[^a-zA-Z]", " ", str(text))

# apply the function to each row of the "description" column
df["description"] = df["description"].apply(remove_non_alphabetic)

In [ ]:
df["description"] = df["description"].apply(lambda x: clean_text(x))

### Cleaning the Categories Column

In [ ]:
import string

# Create a translation table for punctuation removal
translator = str.maketrans(' ', ' ', string.punctuation)

# Apply the translation table to the 'Review' column of the data frame
df['category'] = df['category'].str.translate(translator)

In [ ]:
df.drop('categories', axis='columns')

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,description,category
0,0,0,Footfall,en,706,hostile extraterrestrial known fithp invade ea...,fiction
1,1,1,The Reader,en,224,hailed coiled eroticism moral claim make upon ...,fiction
2,2,2,Hidden Star (The Star Series),en,234,eleven year old nolitye granny used say mess w...,fiction
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,new york time bestseller troubled family vermo...,fiction
4,4,4,The Hours: A Novel,en,295,michael cunningham brings together pulitzer pr...,fiction
...,...,...,...,...,...,...,...
6570,6570,6570,Art of Peace (Shambhala Pocket Classics),en,192.0,inspirational teaching collection show real wa...,philosophy
6571,6571,6571,Autumn Lover,en,416.0,returning wyoming ranch civil war end elyssa s...,fiction
6572,6572,6572,Tales of Two Cities: A Persian Memoir,en,276.0,tale two city engrossing cross cultural memoir...,exile
6573,6573,6573,Heat,en,244.0,bestseller michael arroyo pitching arm throw s...,juvenile fiction


### Merging Categories and Description as major 

In [ ]:
df['Features'] = df['category'].astype(str) + ' ' + df['description'].astype(str)
df.head(5)

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description,category,Features
0,0,0,Footfall,en,706,['Fiction'],hostile extraterrestrial known fithp invade ea...,fiction,fiction hostile extraterrestrial known fithp i...
1,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...
2,2,2,Hidden Star (The Star Series),en,234,['Fiction'],eleven year old nolitye granny used say mess w...,fiction,fiction eleven year old nolitye granny used sa...
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,['Fiction'],new york time bestseller troubled family vermo...,fiction,fiction new york time bestseller troubled fami...
4,4,4,The Hours: A Novel,en,295,['Fiction'],michael cunningham brings together pulitzer pr...,fiction,fiction michael cunningham brings together pul...


### Merge Rating & Book Feature DF

In [ ]:
book_merged = pd.merge(Books, Book_ISBN)

In [ ]:
df_rating = Ratings[Ratings['ISBN'].isin(book_merged['ISBN'])]

In [ ]:
book_merged.head()

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description,category,Features,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0,0,Footfall,en,706,['Fiction'],hostile extraterrestrial known fithp invade ea...,fiction,fiction hostile extraterrestrial known fithp i...,0345323440,Larry Niven,1996,Del Rey Books,http://images.amazon.com/images/P/0345323440.0...,http://images.amazon.com/images/P/0345323440.0...,http://images.amazon.com/images/P/0345323440.0...
1,0,0,Footfall,en,706,['Fiction'],hostile extraterrestrial known fithp invade ea...,fiction,fiction hostile extraterrestrial known fithp i...,0345323475,Larry Niven,1985,Ballantine Books,http://images.amazon.com/images/P/0345323475.0...,http://images.amazon.com/images/P/0345323475.0...,http://images.amazon.com/images/P/0345323475.0...
2,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...,0679442790,Bernhard Schlink,1997,Random House,http://images.amazon.com/images/P/0679442790.0...,http://images.amazon.com/images/P/0679442790.0...,http://images.amazon.com/images/P/0679442790.0...
3,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...,0375707972,Bernhard Schlink,1999,Vintage Books USA,http://images.amazon.com/images/P/0375707972.0...,http://images.amazon.com/images/P/0375707972.0...,http://images.amazon.com/images/P/0375707972.0...
4,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...,0679781307,Carol Brown Janeway,1998,Random House Inc,http://images.amazon.com/images/P/0679781307.0...,http://images.amazon.com/images/P/0679781307.0...,http://images.amazon.com/images/P/0679781307.0...


In [ ]:
df=book_merged

### Review Tokenization

In [ ]:
# Tokenize the data
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

features = df['Features']
tokenized_features = [word_tokenize(feature) for feature in features]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Word Embeddings

In [ ]:
# Word Embeddings Creation

import gensim

# Train the Word2Vec model
model = gensim.models.Word2Vec(tokenized_features, vector_size=250, window=5, min_count=1, workers=8)
# Access the word vectors
word_vectors = model.wv

In [ ]:
word_vectors

### Representing Description into Vectors

In [ ]:
df['Features']

0       fiction hostile extraterrestrial known fithp i...
1       fiction hailed coiled eroticism moral claim ma...
2       fiction eleven year old nolitye granny used sa...
3       fiction new york time bestseller troubled fami...
4       fiction michael cunningham brings together pul...
                              ...                        
6570    philosophy inspirational teaching collection s...
6571    fiction returning wyoming ranch civil war end ...
6572    exile tale two city engrossing cross cultural ...
6573    juvenile fiction bestseller michael arroyo pit...
6574    bean fictitious character card orson scott car...
Name: Features, Length: 6575, dtype: object

In [ ]:
desc_array = []
for feature in df['Features']:
  word_vector = np.zeros((1, 250))
  for word in word_tokenize(feature):
    word_vector += word_vectors[word]
  desc_array.append(word_vector) 

In [ ]:
desc_array = np.array(desc_array)
desc_array = desc_array.reshape(-1,250)
desc_array.shape

(6575, 250)

In [ ]:
desc_array

array([[  2.58355247,  -0.11520911,   2.59334551, ...,  -1.24106617,
         -1.58012329,   1.02832523],
       [ 23.52885126,   1.04008594,  11.12106716, ...,  -8.84684407,
        -10.43388766,   4.80989228],
       [ 38.76853302,  -0.69275227,  18.52786057, ..., -12.82376173,
        -17.01424097,   6.32691999],
       ...,
       [ 26.58901371,  -5.53397005,  21.24578739, ..., -10.97112045,
        -14.56070025,  12.52903819],
       [ 46.19840573,   1.9188477 ,  28.90117656, ..., -15.79473742,
        -20.38117864,   9.85971584],
       [ 18.73709538,  -2.87484076,  23.15604157, ...,  -7.32082503,
        -10.53556924,  10.57495802]])

### Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(desc_array,desc_array).flatten()

In [ ]:
similarities

array([1.        , 0.98017333, 0.97709128, ..., 0.98972698, 0.95777007,
       1.        ])

## Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['Features'])

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
df2 = df.reset_index()
indices = pd.Series(df2.index, index=df2['ISBN'])

### Personlized Recommendation for User ID

In [ ]:
def recommend_books(ISBN, cosine_sim):
    # Get the index of the books that matches the ISBN
    idx = indices[ISBN]

    # Get the pairwsie similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    

    # Get the scores of the most similar books
    sim_scores = sim_scores[1:21]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top most similar books
    return df2[['Book_Title','ISBN']].iloc[book_indices], sim_scores

In [ ]:
def get_recommendations(userid, cosine_sim):
    #userid = str(userid)
    ISBN = df_rating['ISBN'].loc[df_rating[df_rating['User-ID'] == userid]['Book-Rating'].idxmax()]

    recommendation, sim_score = recommend_books(ISBN, cosine_sim)

    df_recommend = pd.DataFrame(recommendation)
    df_recommend['Cosine_Similarity'] = sim_score
    df_recommend['Cosine_Similarity'] = df_recommend['Cosine_Similarity'].apply(lambda x: x[1])

    return df_recommend

In [ ]:
df_recommend = get_recommendations(8, cosine_sim2)
df_recommend=df_recommend.drop_duplicates(subset='Book_Title')
df_recommend[['Book_Title','ISBN']]

,Book_Title,ISBN
12698,Flu : The Story Of The Great Influenza Pandemic,0743203984
5704,Outbreak,042510687X
8667,Fatherland,0061006629
1910,The Illustrated Man (Grand Master Editions),055327449X
10301,Farewell to Manzanar: A True Story of Japanese...,0553272586


### Recommendation Based on ISBN

In [ ]:
# Define a function to get the top K similar books for each book
def get_similar_books(ISBN, cosine_sim, k):

    # Get the row index of the book in the cosine similarity matrix
    idx = book_merged[book_merged['ISBN'] == ISBN].index[0]

    # Get the pairwise cosine similarity scores for the book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the book indices of the top K similar books
    book_indices = [i[0] for i in sim_scores[1:k+1]]

    # Return the top K similar books
    return df2[['Book_Title','ISBN']].iloc[book_indices]

In [ ]:
df_recommend = get_similar_books("0743203984", cosine_sim2,20)
df_recommend=df_recommend.drop_duplicates(subset='Book_Title')
df_recommend

,Book_Title,ISBN
12698,Flu : The Story Of The Great Influenza Pandemic,0743203984
5704,Outbreak,042510687X
8667,Fatherland,0061006629
1910,The Illustrated Man (Grand Master Editions),055327449X
10301,Farewell to Manzanar: A True Story of Japanese...,0553272586
7820,Spandau Phoenix,0451179803
6632,The Years of Rice and Salt,0553580078
9406,My American Journey,0345407288
7154,The Demon in the Freezer,0345466632
7736,The Demon in the Freezer : A True Story,0375508562


## Coverage Evaluation

#### k=10

In [ ]:
# Define recommendation function
def get_recommendations(ISBN, cosine_similarities=cosine_sim2, indices=indices):
    # Get index of book
    idx = indices[ISBN]

    # Get pairwise similarity scores
    sim_scores = list(enumerate(cosine_sim2[idx]))

    # Sort by similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get top 10 most similar books
    sim_scores = sim_scores[1:11]

    # Get book indices
    book_indices = [i[0] for i in sim_scores]

    # Return recommended book ISBNs
    return df[['Book_Title','ISBN']].iloc[book_indices]

In [ ]:
# Get all unique book titles in the dataset
all_books = set(df['ISBN'])

# Define function to get recommended books for all titles
def get_all_recommendations(cosine_similarities=cosine_sim2, indices=indices):
    all_recommendations = []
    for title in all_books:
        recommendations = get_recommendations(title, cosine_sim2, indices)
        all_recommendations.extend(list(recommendations))
    return all_recommendations

# Get all recommended books
all_recommendations = set(get_all_recommendations())

# Calculate coverage
coverage = len(all_recommendations) / len(all_books) * 100

print(f"Coverage of content-based system: {coverage:.2f}%")

Coverage of content-based system with only Book data: 88.55%
